In [ ]:
import sys
# !{sys.executable} -m pip install tldextract

<div style='text-align:center;'>
<h1>BioPortal</h1>
<h2>distance (khops) matrices</h2>
</div>

In [1]:
__author__ = "Lisette Espin-Noboa"
__copyright__ = "Copyright 2018, HopRank"
__credits__ = ["Florian Lemmerich", "Markus Strohmaier", "Simon Walk", "Mark Musen"]
__license__ = "GPL"
__version__ = "1.0.3"
__maintainer__ = "Lisette Espin-Noboa"
__email__ = "Lisette.Espin@gesis.org"
__status__ = "Developing"

########################################################################################
# Warnings
########################################################################################
import warnings
warnings.filterwarnings("ignore", message="numpy.dtype size changed")
warnings.filterwarnings("ignore", message="numpy.ufunc size changed")

<h2>Dependencies</h2>

In [2]:
########################################################################################
# System
########################################################################################
import os
import importlib
import numpy as np
import pandas as pd
import seaborn as sns
import networkx as nx
from collections import Counter
from joblib import Parallel, delayed
import matplotlib as mpl
import matplotlib.pyplot as plt 
from collections import defaultdict

########################################################################################
# Local Dependencies
########################################################################################
%reload_ext autoreload
%autoreload 2
from org.gesis.libs.bioportal.ontology import Ontology
from org.gesis.libs.bioportal.transition import Transition
from org.gesis.libs.bioportal.clickstream import NAVITYPES
from org.gesis.libs.bioportal.clickstream import load_clickstream
from org.gesis.libs.bioportal.clickstream import preprocess_clickstream
from org.gesis.libs.bioportal.clickstream import preprocess_sessions
from org.gesis.libs.bioportal.clickstream import clean_clickstream
from org.gesis.libs.bioportal.submission import get_submissions
from org.gesis.libs.utils import log
from org.gesis.libs.utils import weighted_to_undirected
from org.gesis.libs.utils import to_symmetric

########################################################################################
# Plot's style
########################################################################################
sns.set_context(context="paper", font_scale=1.5)
sns.set_style("whitegrid")

<h2>Constants</h2>

In [3]:
SM = 'submission'
SUBMISSIONS_FN = '/bigdata/lespin/datasets/bioportal/submissions.json'

ON = 'ontology'
ONTOLOGY_ROOT = '/bigdata/lespin/datasets/bioportal/ontologies'
ONTOLOGY_GRAPH_OUTPUT = '/bigdata/lespin/bioportal/ontologies/graph'
ONTOLOGY_ADJACENCY_OUTPUT = '/bigdata/lespin/bioportal/ontologies/matrix'
ONTOLOGY_NODES_OUTPUT = '/bigdata/lespin/bioportal/ontologies/nodes'
ONTOLOGY_HOPS_OUTPUT = '/bigdata/lespin/bioportal/ontologies/hops'
ONTOLOGY_DISTANCE_OUTPUT = '/bigdata/lespin/bioportal/ontologies/distances'

CS = 'clickstream'
CS_FN = 'BP_webpage_requests_<YEAR><POSTFIX>.csv.bz2'
CLICKSTREAM_ROOT = '/bigdata/lespin/datasets/bioportal/clickstream/'
CLICKSTREAM_DF = '/bigdata/lespin/bioportal/clickstreams'

TR = 'transitions'
TRANS_GRAPH_OUTPUT = '/bigdata/lespin/bioportal/transitions/graph'
TRANS_ADJACENCY_OUTPUT = '/bigdata/lespin/bioportal/transitions/matrix'
TRANS_NODES_OUTPUT = '/bigdata/lespin/bioportal/transitions/nodes'
LCC_TRANS_GRAPH_OUTPUT = '/bigdata/lespin/bioportal/transitions_in_lcc/graph'
LCC_TRANS_ADJACENCY_OUTPUT = '/bigdata/lespin/bioportal/transitions_in_lcc/matrix'
LCC_TRANS_NODES_OUTPUT = '/bigdata/lespin/bioportal/transitions_in_lcc/nodes'

SU = 'summary'
LOG_OUTPUT = '/bigdata/lespin/bioportal/logs'
PLOTS_OUTPUT = '/bigdata/lespin/bioportal/plots'
SUMMARY_OUTPUT = '/bigdata/lespin/bioportal/summary'
YEAR = '2015'

LAG_BREAK_IN_MINUTES = 60
MIN_SESSION_LENGTH = 2
MIN_REQ = 1000
MIN_TRANSITIONS = 1000
MAXKHOPS = 30

<div style='text-align:center;'>
<h1>Distance matrices</h1>
</div>

In [5]:
print('loading submissions...')
submissions_dict = get_submissions(SUBMISSIONS_FN)

print('loading ontologies...')
fn = os.path.join(SUMMARY_OUTPUT,'lcc_summary_ontologies_transitions_{}.{}'.format(YEAR,'csv'))
df_onto = pd.read_csv(fn, index_col=False)
tmp = df_onto.loc[df_onto["T'"] >= MIN_TRANSITIONS].copy()
del(df_onto)

print('generating distance matrices for {} ontologies...'.format(tmp.ontology.nunique()))
for index,row in tmp.sort_values("E'", ascending=True).iterrows():
    onto_name = row['ontology']
    
    if onto_name not in ['LOINC']:
        continue

    submission = submissions_dict[onto_name][YEAR]
    onto = Ontology(name=onto_name, year=YEAR, submission_id=submission['submissionId'], root_folder=ONTOLOGY_ROOT)
    m = onto.create_distance_matrix(path=ONTOLOGY_DISTANCE_OUTPUT, hopspath=ONTOLOGY_HOPS_OUTPUT, lcc=True)
    print("max khop (max-diameter): {}".format(m.max()))    
    del(m)


loading submissions...
2019-02-15 19:47:43	/bigdata/lespin/datasets/bioportal/submissions.json loaded!
2019-02-15 19:47:43	- 642 ontologies
2019-02-15 19:47:43	- 13 years
loading ontologies...
generating distance matrices for 12 ontologies...
LCC_LOINC_2015_9HOP.npz
LCC_LOINC_2015_1HOP.npz
LCC_LOINC_2015_4HOP.npz
LCC_LOINC_2015_5HOP.npz
LCC_LOINC_2015_2HOP.npz
LCC_LOINC_2015_7HOP.npz
LCC_LOINC_2015_3HOP.npz
LCC_LOINC_2015_8HOP.npz
LCC_LOINC_2015_12HOP.npz
LCC_LOINC_2015_10HOP.npz
LCC_LOINC_2015_11HOP.npz
LCC_LOINC_2015_6HOP.npz
LCC_LOINC_2015_13HOP.npz
file:LCC_LOINC_2015_9HOP.npz, k:9
2019-02-15 19:47:44	/bigdata/lespin/bioportal/ontologies/hops/LCC_LOINC_2015_9HOP.npz loaded!
file:LCC_LOINC_2015_1HOP.npz, k:1
2019-02-15 19:47:45	/bigdata/lespin/bioportal/ontologies/hops/LCC_LOINC_2015_1HOP.npz loaded!
file:LCC_LOINC_2015_4HOP.npz, k:4
2019-02-15 19:49:46	/bigdata/lespin/bioportal/ontologies/hops/LCC_LOINC_2015_4HOP.npz loaded!
file:LCC_LOINC_2015_5HOP.npz, k:5
2019-02-15 19:51:53	/bi

In [6]:
del(onto)